In [15]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Dense, Dropout
from PIL import Image,ImageOps,ImageEnhance
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint , ReduceLROnPlateau
import os

In [16]:
le = LabelEncoder()

In [17]:
def rotate(img,angle):
    return img.rotate(angle=angle)

def flip(img):
    return ImageOps.mirror(img)

def change_brightness(img,factor = 1.2):
    enhancer = ImageEnhance.Brightness(img)
    return enhancer.enhance(factor)

In [24]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
le.fit(['paper', 'rock', 'scissors'])

# Move this line outside the loop
paths = [] 
# The loop will now only add paths to this single list
for hand in ['paper', 'rock', 'scissors']:
    label = hand
    class_index = le.transform([hand])[0]
    class_label = to_categorical(class_index, num_classes=3)
    
    # The paths variable is no longer reset here
    for r, d, f in os.walk(os.path.join('C:\\Users\\shahe\\Desktop\\ML\\projects\\datasets\\rock\\train', hand)):
        for file in f:
            if file.lower().endswith(".png"):
                paths.append(os.path.join(r, file))

# This loop will now process all the paths in the 'paths' list
data = []
labels = []

for path in paths:
    img = Image.open(path).convert('RGB').resize((64, 64))
    img = np.array(img).astype('float32') / 255.0
    if img.shape == (64, 64, 3):
        data.append(img)
        # You also need to get the labels for the data here
        # using the folder name (hand) from the path
        hand = os.path.basename(os.path.dirname(path))
        label_index = le.transform([hand])[0]
        labels.append(label_index)

le1 = LabelEncoder()
encoded = le1.fit_transform(labels)
result = to_categorical(encoded, num_classes=3)

data = np.array(data)
result = np.array(result)

print('Data shape:', data.shape)
print('Result shape:', result.shape)

Data shape: (7324, 64, 64, 3)
Result shape: (7324, 3)


In [25]:
from sklearn.utils import shuffle
data, result = shuffle(data, result, random_state=42)

In [26]:
result = result.reshape(-1, 3)
result.shape

(7324, 3)

In [27]:
# Assuming 'data' and 'result' are the variables you created
# ... (your image loading and processing code)

print(f"Data shape: {data.shape}")
print(f"Result shape: {result.shape}")

# Add a check to prevent the error
if len(data) > 0:
    xtrain, xtest, ytrain, ytest = train_test_split(data, result, test_size=0.2, random_state=42, stratify=result)
    print("Data split successfully!")
else:
    print("Error: The 'data' array is empty. Check your file loading process and file paths.")

Data shape: (7324, 64, 64, 3)
Result shape: (7324, 3)
Data split successfully!


In [28]:
import collections

flat_y = np.ravel(ytrain)

print("Class distribution in training set:")
print(collections.Counter(flat_y))

Class distribution in training set:
Counter({np.float64(0.0): 11718, np.float64(1.0): 5859})


In [29]:
print("Train:", np.sum(ytrain, axis=0))
print("Test:", np.sum(ytest, axis=0))

Train: [1926. 1954. 1979.]
Test: [482. 488. 495.]


In [30]:
print("ytrain shape:", ytrain.shape)
print("First 5 ytrain values:\n", ytrain[:5])

ytrain shape: (5859, 3)
First 5 ytrain values:
 [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [31]:
print(ytrain[0])

[0. 1. 0.]


In [32]:
from sklearn.utils.class_weight import compute_class_weight

# Convert one-hot to label indices
y_train_labels = np.argmax(ytrain, axis=1)

# Compute weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_labels), y=y_train_labels)
class_weight_dict = dict(enumerate(class_weights))

print("Class weights:", class_weight_dict)


Class weights: {0: np.float64(1.014018691588785), 1: np.float64(0.9994882292732856), 2: np.float64(0.9868620515411825)}


In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, regularizers

model = Sequential()

model.add(layers.Conv2D(16, (3, 3), padding='same',input_shape=(64,64,3))) 
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(16,(3,3),activation='relu',padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(32, (3, 3),activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(32, (3, 3),activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3, 3),activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3),activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Conv2D(64, (3, 3),activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3),activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.3))


model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax')) 



c:\Users\shahe\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [34]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 64, 64, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64, 64, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8, 8, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 8, 8, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             

 Total params: 278,227 (1.06 MB)

 Trainable params: 277,875 (1.06 MB)

 Non-trainable params: 352 (1.38 KB)

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\shahe\Desktop\ML\projects\datasets\rock\train",
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)


Found 7324 images belonging to 3 classes.


In [40]:


val_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    r"C:\Users\shahe\Desktop\ML\projects\datasets\rock\train",
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

Found 7324 images belonging to 3 classes.


In [41]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
loss_fn = CategoricalCrossentropy(label_smoothing=0.1)
optimizer = Adam(learning_rate=1e-3)

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

In [42]:
print("ytrain shape:", ytrain.shape)  # Should be (num_samples, 3)
print(ytrain[:5])  # Should look like [0 1 0], etc.


ytrain shape: (5859, 3)
[[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [43]:
checkpoint = ModelCheckpoint(
    filepath='best_model.h5',  
    monitor='val_loss',         
    save_best_only=True,        
    verbose=1
)

In [44]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    verbose=1
)

In [45]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

In [46]:
print("xtrain shape:", xtrain.shape)
print("ytrain shape:", ytrain.shape)


xtrain shape: (5859, 64, 64, 3)
ytrain shape: (5859, 3)


In [47]:
model.fit(
    train_generator,
    validation_data=(xtest, ytest),
    epochs=100,
    class_weight=class_weight_dict,
    callbacks=[checkpoint, early_stop, reduce_lr],
    shuffle=True
)

c:\Users\shahe\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step - accuracy: 0.3443 - loss: 1.4748
Epoch 1: val_loss improved from inf to 1.09890, saving model to best_model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 41s 300ms/step - accuracy: 0.3445 - loss: 1.4724 - val_accuracy: 0.3331 - val_loss: 1.0989 - learning_rate: 0.0010
Epoch 2/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.4783 - loss: 1.0222
Epoch 2: val_loss did not improve from 1.09890
115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 273ms/step - accuracy: 0.4787 - loss: 1.0218 - val_accuracy: 0.3331 - val_loss: 1.2254 - learning_rate: 0.0010
Epoch 3/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - accuracy: 0.6430 - loss: 0.8421
Epoch 3: val_loss did not improve from 1.09890
115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 276ms/step - accuracy: 0.6433 - loss: 0.8418 - val_accuracy: 0.3331 - val_loss: 1.2058 - learning_rate: 0.0010
Epoch 4/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.7598 - loss: 0.7094
Epoch 4: val_loss improved from 1.09890 to 1.05951, saving model to best_model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 269ms/step - accuracy: 0.7598 - loss: 0.7093 - val_accuracy: 0.4396 - val_loss: 1.0595 - learning_rate: 0.0010
Epoch 5/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.7889 - loss: 0.6544
Epoch 5: val_loss improved from 1.05951 to 0.79403, saving model to best_model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 271ms/step - accuracy: 0.7890 - loss: 0.6543 - val_accuracy: 0.6635 - val_loss: 0.7940 - learning_rate: 0.0010
Epoch 6/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.8086 - loss: 0.6189
Epoch 6: val_loss improved from 0.79403 to 0.70947, saving model to best_model.h5


...

115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 266ms/step - accuracy: 0.9884 - loss: 0.3522 - val_accuracy: 0.9932 - val_loss: 0.3151 - learning_rate: 2.5000e-04
Epoch 34/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.9876 - loss: 0.3519
Epoch 34: val_loss did not improve from 0.31514
115/115 ━━━━━━━━━━━━━━━━━━━━ 30s 263ms/step - accuracy: 0.9875 - loss: 0.3519 - val_accuracy: 0.9932 - val_loss: 0.3222 - learning_rate: 2.5000e-04
Epoch 35/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step - accuracy: 0.9865 - loss: 0.3535
Epoch 35: val_loss did not improve from 0.31514
115/115 ━━━━━━━━━━━━━━━━━━━━ 30s 263ms/step - accuracy: 0.9865 - loss: 0.3534 - val_accuracy: 0.9877 - val_loss: 0.3245 - learning_rate: 2.5000e-04
Epoch 36/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.9858 - loss: 0.3508
Epoch 36: val_loss did not improve from 0.31514

Epoch 36: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
115/115 ━━━━━━━━━━━━━━━━━━━━ 30s 264ms/step - accuracy: 0.9858 -

115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 269ms/step - accuracy: 0.9868 - loss: 0.3503 - val_accuracy: 0.9945 - val_loss: 0.3121 - learning_rate: 1.2500e-04
Epoch 40/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step - accuracy: 0.9924 - loss: 0.3430
Epoch 40: val_loss did not improve from 0.31208
115/115 ━━━━━━━━━━━━━━━━━━━━ 33s 289ms/step - accuracy: 0.9924 - loss: 0.3430 - val_accuracy: 0.9918 - val_loss: 0.3155 - learning_rate: 1.2500e-04
Epoch 41/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.9910 - loss: 0.3436
Epoch 41: val_loss did not improve from 0.31208
115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 273ms/step - accuracy: 0.9910 - loss: 0.3436 - val_accuracy: 0.9884 - val_loss: 0.3196 - learning_rate: 1.2500e-04
Epoch 42/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step - accuracy: 0.9907 - loss: 0.3421
Epoch 42: val_loss did not improve from 0.31208

Epoch 42: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
115/115 ━━━━━━━━━━━━━━━━━━━━ 34s 297ms/step - accuracy: 0.9907 - 

115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 281ms/step - accuracy: 0.9923 - loss: 0.3428 - val_accuracy: 0.9973 - val_loss: 0.3111 - learning_rate: 6.2500e-05
Epoch 45/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.9907 - loss: 0.3422
Epoch 45: val_loss did not improve from 0.31109
115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 274ms/step - accuracy: 0.9907 - loss: 0.3422 - val_accuracy: 0.9959 - val_loss: 0.3119 - learning_rate: 6.2500e-05
Epoch 46/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.9893 - loss: 0.3456
Epoch 46: val_loss improved from 0.31109 to 0.30927, saving model to best_model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 276ms/step - accuracy: 0.9894 - loss: 0.3456 - val_accuracy: 0.9973 - val_loss: 0.3093 - learning_rate: 6.2500e-05
Epoch 47/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.9913 - loss: 0.3441
Epoch 47: val_loss improved from 0.30927 to 0.30869, saving model to best_model.h5


115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 276ms/step - accuracy: 0.9913 - loss: 0.3441 - val_accuracy: 0.9966 - val_loss: 0.3087 - learning_rate: 6.2500e-05
Epoch 48/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.9950 - loss: 0.3367
Epoch 48: val_loss did not improve from 0.30869
115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 272ms/step - accuracy: 0.9950 - loss: 0.3367 - val_accuracy: 0.9966 - val_loss: 0.3104 - learning_rate: 6.2500e-05
Epoch 49/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.9905 - loss: 0.3415
Epoch 49: val_loss did not improve from 0.30869
115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 271ms/step - accuracy: 0.9905 - loss: 0.3415 - val_accuracy: 0.9973 - val_loss: 0.3109 - learning_rate: 6.2500e-05
Epoch 50/100
115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.9900 - loss: 0.3409
Epoch 50: val_loss did not improve from 0.30869

Epoch 50: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 271ms/step - accuracy: 0.9900 -

In [48]:
from sklearn.metrics import classification_report
import numpy as np

# Evaluate on test set
loss, acc = model.evaluate(xtest, ytest, verbose=1)
print(f"Test Accuracy: {acc:.4f}")
print(f"Test Loss: {loss:.4f}")

# Predict class probabilities
y_pred_probs = model.predict(xtest)

# Convert one-hot to class indices
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(ytest, axis=1)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes))


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9971 - loss: 0.3089
Test Accuracy: 0.9973
Test Loss: 0.3093
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       482
           1       0.99      1.00      1.00       488
           2       1.00      1.00      1.00       495

    accuracy                           1.00      1465
   macro avg       1.00      1.00      1.00      1465
weighted avg       1.00      1.00      1.00      1465



In [49]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Optional: Load best weights
model.load_weights('best_model.h5')

# Step 1: Recompile with a lower learning rate
fine_tune_lr = 1e-4  # Or 5e-5
model.compile(
    optimizer=Adam(learning_rate=fine_tune_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 2: Set up callbacks (reusing if already defined)
checkpoint = ModelCheckpoint("finetuned_model.keras", save_best_only=True, monitor="val_loss", mode="min")
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2)

# Step 3: Fit again with fewer epochs
history_finetune = model.fit(
    train_generator,
    validation_data=(xtest, ytest),
    epochs=20,
    callbacks=[checkpoint, early_stop, reduce_lr],
    shuffle=True
)


Epoch 1/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 36s 284ms/step - accuracy: 0.9899 - loss: 0.0700 - val_accuracy: 0.9939 - val_loss: 0.0235 - learning_rate: 1.0000e-04
Epoch 2/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 277ms/step - accuracy: 0.9902 - loss: 0.0374 - val_accuracy: 0.9952 - val_loss: 0.0187 - learning_rate: 1.0000e-04
Epoch 3/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 32s 274ms/step - accuracy: 0.9882 - loss: 0.0330 - val_accuracy: 0.9925 - val_loss: 0.0213 - learning_rate: 1.0000e-04
Epoch 4/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 31s 273ms/step - accuracy: 0.9895 - loss: 0.0313 - val_accuracy: 0.9939 - val_loss: 0.0206 - learning_rate: 1.0000e-04
Epoch 5/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 34s 292ms/step - accuracy: 0.9902 - loss: 0.0309 - val_accuracy: 0.9918 - val_loss: 0.0244 - learning_rate: 5.0000e-05
Epoch 6/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 38s 334ms/step - accuracy: 0.9873 - loss: 0.0354 - val_accuracy: 0.9959 - val_loss: 0.0154 - learning_rate: 5.0000e-05
Epoch 7/20
115/115 ━━━━━━━━━━━━━━━━━━━━ 34s 29

In [50]:
model.load_weights("finetuned_model.keras")

loss, acc = model.evaluate(xtest, ytest)
print(f"Fine-tuned Test Accuracy: {acc:.4f}")
print(f"Fine-tuned Test Loss: {loss:.4f}")


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9968 - loss: 0.0147
Fine-tuned Test Accuracy: 0.9959
Fine-tuned Test Loss: 0.0138


In [51]:
model.save("rps_model.h5")